In [13]:
# This notebook is to get Future Predicted values
#Loading the libraries
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout   
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from sklearn.preprocessing import MinMaxScaler
from keras import optimizers

# Reading csv file
#customers timeseries updated has data for 100 customers
df = pd.read_csv("customers_timeseries_updated.csv")
#only using required columns for modelling
df = df[['client_debtor_number','dates','fv_cost']]
# number of unique customers in the data
len(df['client_debtor_number'].unique())

/Users/anusha/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


101

In [14]:
# subsetting one customer data from the set
df_1015130 = df[df['client_debtor_number'] == 1015193]
del df_1015130['client_debtor_number']
print(df_1015130.head())

        dates  fv_cost
0  10/07/2010      0.0
1  11/07/2010      0.0
2  12/07/2010      0.0
3  13/07/2010      0.0
4  14/07/2010      0.0


In [16]:
# splitting train and test datasets 
#df_1015130['dates'] = df_1015130.index.values
df_1015130['dates'] = pd.to_datetime(df_1015130['dates'],format= '%d/%m/%Y')
#weekly data
week_data = df_1015130.set_index('dates').resample('1W').mean()
week_data['dates'] = week_data.index.values
#training 2010-2017, testing on 2018 data\
#train_x = week_data[week_data['dates'].str.contains('/2018') == False]
#test_x = week_data[week_data['dates'].str.contains('/2018') == True]
train_x = week_data[~(week_data['dates'].dt.year == 2018)]
test_x = week_data[(week_data['dates'].dt.year == 2018)]
#resetting index values
train_x.reset_index(drop=True,inplace=True)
test_x.reset_index(drop=True,inplace=True)

#sorting data based on dates
train_x['dates'] = pd.to_datetime(train_x['dates'],format= '%d/%m/%Y')
train_x.sort_values(by='dates',inplace=True)
test_x['dates'] = pd.to_datetime(test_x['dates'],format= '%d/%m/%Y')
test_x.sort_values(by='dates',inplace=True)

/Users/anusha/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/anusha/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/anusha/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas

In [17]:
# lag_feature function is used to get new column in the dataframe with 
#lagged data,number of lags can be given as function parameter 
def lag_feature(df, lag=1):
    if not type(df) == pd.DataFrame:
        df = pd.DataFrame(df, columns=['fv_cost'])   
    def rename_lag(ser, j):
        ser.name = ser.name + f'_{j}'
        return ser        
    # add a column lagged by `i` steps
    for i in range(1, lag + 1):
        df = df.join(df.fv_cost.shift(i).pipe(rename_lag, i))
    df.dropna(inplace=True)
    return df

# Prepare training data function is used to scale the fv_cost values 
# between -1 to 1 and calls lag_feature to create lag columns 
def prepare_training_data(series_data, lag):
    " Converts a series of data into a lagged, scaled sample."
    # scale training data
    scaler = MinMaxScaler(feature_range=(-1, 1))
    #when diff is used don't use values
    cost_vals = scaler.fit_transform(series_data.values.reshape(-1, 1))   
    #cost_vals = scaler.fit_transform(series_data.reshape(-1, 1))
    # convert series to lagged features
    cost_lagged = lag_feature(cost_vals, lag=lag)
    # X, y format taking the first column (original time series) to be the y
    X = cost_lagged.drop('fv_cost', axis=1).values
    y = cost_lagged.fv_cost.values
    
    # keras expects 3 dimensional X
    X = X.reshape(X.shape[0], 1, X.shape[1])   
    return X, y, scaler

#Difference function is get lag difference values
def difference(dataset):
    interval=1
    diff = list()
    for i in range(interval, len(dataset)):
        #print ("iteration ",i)
        value = (dataset.iloc[i] - dataset.iloc[i - interval])/(1+abs(dataset.iloc[i - interval]))
        diff.append(value)
    return diff

# if you are using difference lag values use this code
#diff_train = np.array(difference(train_x['fv_cost']))
#diff_test = np.array(difference(test_x['fv_cost']))
#train_x_cust,train_y_cust, scalar_train = prepare_training_data(diff_train, 3)
#test_x_cust,test_y_cust,scalar_test = prepare_training_data(diff_test, 3)
# preparing data on train and test fv_cost column values
train_x_cust,train_y_cust, scalar_train = prepare_training_data(train_x['fv_cost'], 3)
test_x_cust,test_y_cust,scalar_test = prepare_training_data(test_x['fv_cost'], 3)
print(train_x_cust.shape)
print(train_y_cust.shape)


(415, 1, 3)
(415,)


In [18]:
# Model description
# lag is number of lags used to prepare the data
lag =  3
# model parameters
num_neurons = 50 #number of neurons/nodes for the layer
# actually we can give data in batches to the model
# number of samples in the data should be divisible by batch size
# we are giving all data for a customer as one batch
batch_size = 1  
# input_shape as required by LSTM layer
batch_input_shape=(batch_size, 1, lag)
# dropout rate used in dropout layer
dropout_rate =0.2
# instantiate a sequential model
model = Sequential()
#add convolution layer
# input_shape=(3,1) 3 lags columns and 1 y value
# when strides>1 you cannot have dilation>1
# activation_func options ['softmax', 'softplus', 'softsign', 'relu', 
#                   'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
# in conv1D layer we can modify filters,kernel_size,strides and activation
# parameters
model.add(Conv1D(filters=num_neurons,batch_size=1, kernel_size=3, strides=3, 
                 padding="same",activation='linear',dilation_rate=1, 
                 input_shape=(1, 3),data_format='channels_first'))
# maxpooling layers tries to reduce the features by taking maximum value for
# window/pool size selected,strides and pool_size can be changed
model.add(MaxPooling1D(pool_size=3,strides=3, padding="same"))
model.add(Dropout(dropout_rate))
# add LSTM layer - stateful MUST be true here in 
# order to learn the patterns within a series
model.add(LSTM(units=num_neurons, 
              batch_input_shape=batch_input_shape, return_sequences=False,# as we only want last hidden output 
              stateful=True))
# output layer
model.add(Dense(1,activation='linear'))
# compile the model with required loss and optimizer function
adam = optimizers.Adam(lr=0.01, decay=0.01)
model.compile(loss='mean_squared_error', optimizer=adam)

In [19]:
#Fit the model
model.fit(train_x_cust, train_y_cust, epochs=10, batch_size=batch_size, verbose=2, shuffle=False)

Epoch 1/10
 - 5s - loss: 0.0519
Epoch 2/10
 - 4s - loss: 0.0653
Epoch 3/10
 - 4s - loss: 0.0290
Epoch 4/10
 - 4s - loss: 0.0258
Epoch 5/10
 - 5s - loss: 0.0156
Epoch 6/10
 - 5s - loss: 0.0128
Epoch 7/10
 - 5s - loss: 0.0149
Epoch 8/10
 - 5s - loss: 0.0124
Epoch 9/10
 - 5s - loss: 0.0121
Epoch 10/10
 - 5s - loss: 0.0116


In [20]:
# Evaluate model performance
# default batch size is 32 so we need to give batch size explicitly if we want different batch size or 
# input size should be divisible by batch size for stateful LSTM
trainScore = model.evaluate(train_x_cust, train_y_cust, batch_size=1, verbose=2)
print('Train Score: %.2f MSE (%.2f RMSE)' % (trainScore, sqrt(trainScore)))
testScore = model.evaluate(test_x_cust, test_y_cust, batch_size=1, verbose=2)
print('Test Score: %.2f MSE (%.2f RMSE)' % (testScore, sqrt(testScore)))

print('unscaled test score %f MSE' %(scalar_test.inverse_transform(testScore)))
yhat_act_test = scalar_test.inverse_transform(testScore)
yhat_act_train = scalar_train.inverse_transform(trainScore)
print ("actual test rmse ",sqrt(mean_squared_error(test_x.fv_cost[-(len(yhat_act_test)):], yhat_act_test)))
print ("actual train rmse ",sqrt(mean_squared_error(train_x.fv_cost[-(len(yhat_act_train)):],yhat_act_train)))
#scaler.inverse_transform(testScore)

Train Score: 0.02 MSE (0.15 RMSE)
Test Score: 0.08 MSE (0.28 RMSE)


ValueError: Expected 2D array, got scalar array instead:
array=0.07928849593710968.
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
# predictions of model
trainPredict = model.predict(train_x_cust,batch_size=1)
testPredict = model.predict(test_x_cust,batch_size=1)

In [ ]:
# write predictions into csv
result_prediction = pd.DataFrame()#, original_fv : test_y_cust, prediction_fv :testPredict})
#result_prediction['dates'] = test_x['dates']
result_prediction['original_fv'] = test_y_cust
result_prediction['prediction_fv'] = testPredict

result_prediction.to_csv("predictions_for_client_1015193.csv")

Experimentation on Future values of train dataset

In [ ]:
# predictions for timesteps ahead
# 7,14 ,21 timesteps ahead
#we need atleast 4 elements to prepare training data
#futureElement = testPredict[-4:]
# if using testpredict as futureElement no need to scale data again using MinMaxscalar
futureElement = train_x.fv_cost[-4:]
x=futureElement[-3:]
x=x.reshape(-1, 1)
x.reshape(x.shape[1], 1, x.shape[0])

def timestep_ahead(num,model,futureElement,x):
    #Since we are using train we need to scale the input
    futureElement_x,futureElement_y,scalar = prepare_training_data(futureElement[-4:], 3)
    #print ("shape of fe_x before for loop ",futureElement_x.shape)
    futureElements = []
    unscaled_prediction =[]
    futureElements.append(futureElement.tolist())
    # plus 1 because one value for futureElement_y 
    for i in range(num+1):
       # print(futureElement_x.shape,"shape after preparing data in ",i," iteration")
        #print("future element value after preparing data",futureElement_x)
        futureElement = model.predict(futureElement_x,batch_size=1,verbose=2)
        unscaled_prediction.append(scalar.inverse_transform(futureElement).tolist())
        #print("unscaled prediction ",unscaled_prediction)
        futureElements.append(futureElement.tolist())
        #print("future element before appending x",futureElement)
        futureElement = np.append(x,futureElement)
        x=futureElement[-3:]
        futureElement_x=x.reshape(-1, 1)
        futureElement_x=futureElement_x.reshape(
            futureElement_x.shape[1], 1, futureElement_x.shape[0])
       # print("future element after appending x",futureElement)
        #print("futureElements ",futureElements)     
        #print("iteration ",i,"  complete")     
    return (futureElement[-1],futureElements,unscaled_prediction)


In [ ]:
#comparing future elements of train data with test dataset to check model performance
# change the number to get number of predictions ahead
_7ahead,elements,unscaled_prediction = timestep_ahead(7,model,train_x.fv_cost[-4:],x)
print ("prediction of 7days ahead ",_7ahead,"scaled actual value",test_y_cust[7]," unscaled prediction ",unscaled_prediction[-1]," actual value ",test_x.fv_cost.iloc[7])

In [ ]:
_14ahead,elements,unscaled_prediction = timestep_ahead(14,model,train_x.fv_cost[-4:],x)
print ("prediction of 14days ahead ",_14ahead,"scaled actual value",test_y_cust[14]," unscaled prediction ",unscaled_prediction[-1]," actual value ",test_x.fv_cost.iloc[14])


In [ ]:
_21ahead,elements,unscaled_prediction = timestep_ahead(21,model,train_x.fv_cost[-4:],x)
print ("prediction of 21days ahead ",_21ahead,"scaled actual value",
       test_y_cust[21]," unscaled prediction ",
       unscaled_prediction[-1]," actual value ",test_x.fv_cost.iloc[21])

In [ ]:
_28ahead,elements,unscaled_prediction = timestep_ahead(28,model,train_x.fv_cost[-4:],x)
print ("prediction of 28days ahead ",_28ahead,"scaled actual value",
       test_y_cust[28]," unscaled prediction ",
       unscaled_prediction[-1]," actual value ",test_x.fv_cost.iloc[28])

In [ ]:
_90ahead,elements,unscaled_prediction = timestep_ahead(90,model,train_x.fv_cost[-4:],x)
print ("prediction of 90days ahead ",_90ahead,"scaled actual value",
       test_y_cust[90]," unscaled prediction ",
       unscaled_prediction[-1]," actual value ",test_x.fv_cost.iloc[90])